In [2]:
import pandas as pd
import numpy as np
import graphlab as gl
# pip install --upgrade 
# --no-cache-dir https://get.dato.com/GraphLab-Create/1.8.5/justin@thezecks.com
# /9378-F674-129B-F6B9-F178-B959-E07A-3D81/GraphLab-Create-License.tar.gz


''' File 5: all of the above,
tldr: the above table is all previous tables joined on year and quarter, household_id, and person_id

The incident-level extract file is not the same as the incident record-type file. The incident-level extract file is a file created by prepending
household and person variables to incident records using the YEARQ (YEAR AND QUARTER OF INTERVIEW), IDHH (NCVS ID FOR
HOUSEHOLDS), and IDPER (NCVS ID FOR PERSONS) variables as match keys. For data-year formats, this file has been "bounded"
to contain incidents occurring within the specific calendar year, regardless of when the interview was conducted. Under the collection-year
format, the incident-level extract file is not "bounded" based on when the incidents occurred; rather, it contains incidents reported in 2012,
regardless of when they occurred
'''

In [35]:
df_data = pd.read_table('data/5-Data.tsv')


In [5]:
# dfall.info()
# 9215 rows, 1145 collumns

df_data.head()

,V2001,YEARQ,IDHH,IDPER,V2002,V2003,V2004,V2005,V2006,V2008,...,VICREPWGT151,VICREPWGT152,VICREPWGT153,VICREPWGT154,VICREPWGT155,VICREPWGT156,VICREPWGT157,VICREPWGT158,VICREPWGT159,VICREPWGT160
0,2,2012.1,2501051210759582293728435,250105121075958229372843501,24034,121,25,01051210759582293728,4,35,...,808.246043,4434.171264,2713.832418,4431.329835,2659.149306,2651.307151,4738.553773,2773.348911,4323.180896,844.934676
1,2,2012.1,2501051210759582293728435,250105121075958229372843501,24034,121,25,01051210759582293728,4,35,...,808.246043,4434.171264,2713.832418,4431.329835,2659.149306,2651.307151,4738.553773,2773.348911,4323.180896,844.934676
2,2,2012.1,2501051210759582293728435,250105121075958229372843501,24034,121,25,01051210759582293728,4,35,...,861.844113,4727.533125,2908.610257,4739.329797,2836.045781,2836.421004,5073.342420,2944.663118,4647.547199,902.387830
3,2,2012.1,2501051210759582293728435,250105121075958229372843501,24034,121,25,01051210759582293728,4,35,...,861.844113,4727.533125,2908.610257,4739.329797,2836.045781,2836.421004,5073.342420,2944.663118,4647.547199,902.387830
4,2,2012.1,2501286218428920608853213,250128621842892060885321301,26233,121,25,01286218428920608853,2,13,...,1899.977652,4218.009488,3351.802607,4782.493661,3090.280119,2076.777830,4058.076475,1956.811167,3694.022398,1971.874188


In [10]:
def get_summary(col, targ):
    
    cnts = col.value_counts(normalize=True)
    num_unique = len(cnts)
    top_freq = cnts.values[0]
    top_item = cnts.index[0]
    fraction_missing = col.isnull().mean()
    first_few = ', '.join(list(col.unique()
                               .astype(str)[:max(num_unique, 3)]))
    
    #Get target frequency spread
    #print col, targ
    temp = pd.concat([col, targ],axis=1)
    grouped = temp.groupby(col.name)['target'].mean()
    freq_range = grouped.max() - grouped.min()
    
    return pd.Series(
        {'NumUnique' : num_unique,
         'Top Freq' : top_freq,
         'Top Item' : top_item,
         'Fraction Missing' : fraction_missing,
         'Examples' : first_few,
         'Freq Range' : freq_range
        })

df_cat = df_data
# Summarise each categorical variable
#df_cat.apply(lambda col: get_summary(col, df_data.columns)).T   

In [36]:


#df_data.V4528.value_counts(dropna=False)

In [31]:
def makeNmodels(predict_these):
    
    scores = []
    
    for curCol in predict_these:
        score = doOneModel(curCol)
        
        col_with_score = (curCol, score)
        
        scores.apend(col_with_score)
    
    return scores
        
    

In [37]:
def delFeatures(features):
    for cur_feat in features:
        del df_data[cur_feat]

In [38]:
low_var_features = ['V2001', 'V2009', 'V2027', 'V2028', 'V2029', 'V2030', 'V2031', 'V2109', 
                    'V2110', 'V2112', 'V2114', 'V2115', 'V2123', 'V2131', 'V2142', 'V3001', 
                    'V3027', 'V3051', 'V3057', 'V3060', 'V3069', 'V3082', 'V4001', 'V4319', 
                    'V4320']
                        #'V4528
overpowered_features = ['V4529', 'V4112', 'V4060']

'''
V4528    TYPE OF CRIME CODE (OLD, NCS)
V4529 - TYPE OF CRIME CODE (NEW, NCVS)
V4112 -    INJURIES: RAPE INJURIES  
V4060 - OFFENDER HIT OR ATTACK (ALLOCATED)
'''
delFeatures(low_var_features)
delFeatures(overpowered_features)


In [43]:
def doOneModel(predict):
    
    sf_data = gl.SFrame(df_data)
    
    # graphlab.boosted_trees_classifier.create(dataset, target, features=None, 
    # max_iterations=10, validation_set='auto', class_weights=None, max_depth=6, 
    # step_size=0.3, min_loss_reduction=0.0, min_child_weight=0.1, row_subsample=1.0, 
    # column_subsample=1.0, verbose=True, random_seed=None, metric='auto', **kwargs)
    
    model = gl.boosted_trees_classifier.create(sf_data, predict, features=None, 
            max_iterations=10, validation_set='auto', class_weights=None, max_depth=10, 
            step_size=0.3, min_loss_reduction=0.0, min_child_weight=0.1, row_subsample=1.0, 
            column_subsample=1.0, verbose=True, random_seed=42, metric='auto')
    
    predictions = model.predict(sf_data)
    results = model.evaluate(sf_data)
    print " -> Finished computing model."
    print "\n<------------------------------------------------------------->"
    print "Results for " + str(predict) + " :\n"
    print "<------------------------------------------------------------->"
    return model
    
    

In [41]:
def printResults(model):
    
    print model.get_feature_importance()
    
    print model.summary()

In [44]:
predict = 'V4028'

m_boostedT = doOneModel(predict)
printResults(m_boostedT)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 8786

...

| 5         | 3.901741     | 1.000000          | 0.212676          | 1.000000            | 0.212756            |

| 6         | 5.065287     | 1.000000          | 0.155150          | 1.000000            | 0.155225            |

| 10        | 8.206563     | 1.000000          | 0.045568          | 1.000000            | 0.045606            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+


<------------------------------------------------------------->
Results for V4028 :

<------------------------------------------------------------->
+-------+----------------------+-------+
|  name |        index         | count |
+-------+----------------------+-------+
| V4026 |         None         |   30  |
| V4040 |         None         |   20  |
| V4027 |         None         |   19  |
| V4029 |         None         |   10  |
| V4528 |         None         |   10  |
| V4043 |         None         |   9   |
| V4022 |         None         |   1   |
| V3005 | 88124190309120292953 |   0   |
| V3005 | 88237487453082292828 |   0   |
| V3005 | 88127458509188292904 |   0   |
+-------+----------------------+-------+
[33383 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 8786
Number of featur

In [ ]:
'''
-------------------------------------------------------------
+---------+-------+-------+
|   name  | index | count |
+---------+-------+-------+
|  V4364  |  None |  317  |V4364 - VALUE OF PROPERTY TAKEN
|  V4321  |  None |  245  |VALUE OF CASH TAKEN
|  V4024  |  None |  229  |WHERE DID INCIDENT HAPPEN
Value
Label
In respondent's home or lodging
01 In own dwelling, own attached garage, or enclosed porch (Include illegal entry or attempted illegal entry of same)
02 In detached building on own property, such as a detached garage,..
...

Commercial places
12 Inside restaurant, bar, nightclub
13 Inside other commercial building such as store, bank, gas station
...

Parking lots/garages
15 Commercial parking lot/garage
16 Noncommercial parking lot/garage
17 Apartment/townhouse parking lot/garage School
18 Inside school building
19 On school property (school parking area, play area, school bus, etc.)
...
23 Other
98 Residue
99 (M) Out of universe

| V4140B1 |  None |  142  |VICTIM OF CRIME LEAD TO PROBLEMS JOB, SCHOOLWORK, BOSS, COWORKERS, OR PEERS
|  V4111  |  None |  101  |INJURIES: NONE
|  V4026  |  None |   99  |DID OFFENDER GET INSIDE
|  V4049  |  None |   82  |DID OFFENDER HAVE A WEAPON
|  V4289  |  None |   81  |ATTEMPTED THEFT (ORIGINAL)
|  V4358  |  None |   72  |CASH/WALLET ON PERSON (ORIGINAL)
|  V4028  |  None |   50 | EVIDENCE OF FORCIBLE ENTRY
'''


# Things to try later:
https://dato.com/products/create/docs/graphlab.toolkits.feature_engineering.html#categorical-features

RandomProjection 	Project high-dimensional numeric features into a low-dimensional subspace.

OneHotEncoder 	Encode a collection of categorical features using a 1-of-K encoding scheme.
CountThresholder 	Map infrequent categorical variables to a new/separate category.
CategoricalImputer 	The purpose of this imputer is to fill missing values (None) in data sets
 that have categorical data.

